In [1]:
import subprocess
import os
import sys
sys.path.append('../')
import utils as ul
import copy
import pickle
import gzip
from matplotlib import pyplot as plt
import numpy as np
from scipy.signal import medfilt
import h5py

in_dir = '/data/inspur_disk03/userdir/caoxy/eboss_lya/data/'
lya_hits = pickle.load(gzip.open(in_dir+'lya_hits_cut.pzip'))

dir1 = os.getcwd()
os.chdir('/data/inspur_disk03/userdir/caoxy/eboss_lya/data/2')
dir2 = os.getcwd()

plate_list = lya_hits['plate_hits']
fiber_list = lya_hits['fiber_hits']
mjd_list = lya_hits['mjd_hits']
spec_id_list = lya_hits['spec_id']
peak_id_list = lya_hits['peak_id']
peak_wave_list = lya_hits['peak_wave']
peak_sn_list = lya_hits['peak_sn']


for i in range(plate_list.size):
    this_plate = plate_list[i]
    this_mjd = mjd_list[i]
    this_fiber = fiber_list[i]
    this_spec_id = spec_id_list[i]
    this_peak_id = peak_id_list[i]
    this_peak_wave = peak_wave_list[i]
    this_peak_sn = peak_sn_list[i]
    h5_file = '{}_{}.h5'.format(this_plate,this_mjd)
    with h5py.File(h5_file, 'r') as f:
        arr0 = f['flux']
        flux = arr0[:]
        
        arr1 = f['loglam']
        loglam = arr1[:]
        
        arr2 = f['zans']
        zans = arr2[:]
        
        arr3 = f['newsynflux']
        synflux = arr3[:]
        
        arr4 = f['res_ivar']
        res_ivar = arr4[:]
    
    
    this_wave = 10**loglam
    this_flux = flux[this_spec_id,:]
    this_synflux = synflux[this_spec_id,:]
    this_resflux = this_flux - this_synflux
    this_res_ivar = res_ivar[this_spec_id,:]
    this_sn = np.sqrt(this_resflux**2 * this_res_ivar)
    
    
    hw_sub = 25 #empirically choice
    wave_sub = this_wave[this_peak_id-hw_sub:this_peak_id+hw_sub+1]
    resflux_sub =  this_resflux[this_peak_id-hw_sub:this_peak_id+hw_sub+1]
    res_ivar_sub = this_res_ivar[this_peak_id-hw_sub:this_peak_id+hw_sub+1]
    sn_sub = this_sn[this_peak_id-hw_sub:this_peak_id+hw_sub+1]
    
    #make sure reading process is correct
    #print(zans[this_spec_id])
    #print(this_plate,this_mjd,this_fiber)
    
    #visulization
    from matplotlib import pyplot as plt
    import matplotlib.gridspec as gridspec
    name = r'ID: {}-{}-{}'.format(this_plate,this_mjd,this_fiber)
    hit_wave = r'Hit $\lambda$: {:.2f}'.format(this_peak_wave)
    hit_sn = r'Hit SN: {:.2f}'.format(this_peak_sn)
    textstr = '\n'.join((name,hit_wave,hit_sn))
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    
    #fig, ax = plt.subplots(2,2,figsize=(20,15)) #,sharey=True
    fig = plt.figure(figsize=(30,20),tight_layout=True)
    gs = gridspec.GridSpec(3, 4)
    
    #--------------------------subplot 0,0:
    ax = fig.add_subplot(gs[0, 0:3])
    ax.plot(this_wave,this_flux,color='deepskyblue')
    ax.plot(this_wave, this_synflux,color='red')
    ax.axvline(x=this_peak_wave,color='magenta')
    id_mask = ul.return_mask_region_id(this_res_ivar)
    for this_mask_id in id_mask:
        low = this_mask_id[0]
        high = this_mask_id[1]
        #print(this_wave[low],this_wave[high-1])
        ax.axvspan(this_wave[low], this_wave[high-1], facecolor='g', alpha=0.5)
    #ax[0,0].set_xlim(3700,4800)
    up_lim = this_flux.max()*1.2
    if this_flux.min() < 0:
        low_lim = this_flux.min()*1.2
    else:
        low_lim = -0.1
    ax.set_ylim(low_lim,up_lim)
    
    
    #--------------------------subplot 1,0
    ax = fig.add_subplot(gs[1, 0:2])
    ax.plot(this_wave,this_flux,color='deepskyblue')
    ax.plot(this_wave, this_synflux,color='red')
    ax.axvline(x=this_peak_wave,color='magenta')
    id_mask = ul.return_mask_region_id(this_res_ivar)
    for this_mask_id in id_mask:
        low = this_mask_id[0]
        high = this_mask_id[1]
        #print(this_wave[low],this_wave[high-1])
        ax.axvspan(this_wave[low], this_wave[high-1], facecolor='g', alpha=0.5)
    ax.set_xlim(3700,4800)
    id_tmp = np.where(np.logical_and(this_wave<4800,this_wave>3700))
    up_lim = this_flux[id_tmp].max()*1.2
    if this_flux[id_tmp].min() < 0:
        low_lim = this_flux[id_tmp].min()*1.2
    else:
        low_lim = -0.1
    ax.set_ylim(low_lim,up_lim)
    
    
    #--------------------------subplot 1,1
    ax = fig.add_subplot(gs[1, 2:])
    ax.bar(wave_sub,sn_sub,alpha=0.5)
    ax.plot(wave_sub,sn_sub,color='red')
    ax.axvline(x=this_peak_wave,color='magenta')
    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=10,
            verticalalignment='top', bbox=props)
    wave_, sn_ = ul.return_peak(wave_sub,sn_sub)
    ax.plot(wave_,sn_,'*',color='black',ms=20)
    
    
    #--------------------------subplot 2,0
    ax = fig.add_subplot(gs[2, 0:2])
    _,sn_filter_0,_ = ul.filter_sn(resflux=resflux_sub,resivar=res_ivar_sub,wave=wave_sub,pix_sigma=1.1,kern_size=15)
    ax.bar(wave_sub,sn_filter_0,alpha=0.5)
    ax.plot(wave_sub,sn_filter_0,color='red')
    ax.axvline(x=this_peak_wave,color='magenta')
    ax.text(0.05, 0.95, 'filtered spectrum 1.1', transform=ax.transAxes, fontsize=15,
            verticalalignment='top', bbox=props)
    wave_, sn_ = ul.return_peak(wave_sub,sn_filter_0)
    ax.plot(wave_, sn_,'*',color='black',ms=20)
    
    
    #--------------------------subplot 2,1
    ax = fig.add_subplot(gs[2, 2:])
    _,sn_filter_1,_ = ul.filter_sn(resflux=resflux_sub,resivar=res_ivar_sub,wave=wave_sub,pix_sigma=2.2,kern_size=15)
    ax.bar(wave_sub,sn_filter_1,alpha=0.5)
    ax.plot(wave_sub,sn_filter_1,color='red')
    ax.axvline(x=this_peak_wave,color='magenta')
    ax.text(0.05, 0.95, 'filtered spectrum 2.2', transform=ax.transAxes, fontsize=15,
            verticalalignment='top', bbox=props)
    wave_, sn_ = ul.return_peak(wave_sub,sn_filter_1)
    ax.plot(wave_, sn_,'*',color='black',ms=20)
    
    #--------------------------subplot 0,1
    data_dir = '/data/inspur_disk03/userdir/caoxy/eboss_lya/data/cross_talk/'
    color = ['black','red','cyan']
    ax = fig.add_subplot(gs[0, 3])
    for jj in range(3):
        h5_file = data_dir + '{}_{}_{}_{}.h5'.format(this_plate,this_mjd, this_fiber,jj)
        with h5py.File(h5_file, 'r') as f:
            arr0 = f['flux']
            flux = arr0[:]
            
            arr1 = f['wave']
            wave = arr1[:]
            
            arr2 = f['zans']
            zans = arr2[:]
        ax.plot(wave,flux,color=color[jj])
        ax.set_xlim(this_peak_wave-50,this_peak_wave+50)
        ax.minorticks_on()
    
    #plt.show()
    png_name = '{}-{}-{}'.format(this_plate,this_mjd,this_fiber)
    out_png = '/data/inspur_disk03/userdir/caoxy/eboss_lya/data/4/{}'.format(png_name)
    if not os.path.exists(out_png+'_1.png'):  
        fig.savefig(out_png+'_1.png',dpi=40,bbox_inches='tight')
        plt.close(fig)
        print(out_png+'_1.png','---{} image-----finished!'.format(i))
    else:
        output = subprocess.Popen('ls -d {}*'.format(out_png),stdout=subprocess.PIPE,shell=True)
        lines = output.stdout.read().splitlines()
        print('----lines:',lines)
        k_hits = len(lines)
        fig.savefig(out_png+'_{}.png'.format(k_hits+1),dpi=40,bbox_inches='tight')
        plt.close(fig)
        print(out_png+'_{}.png'.format(k_hits+1),'---{} image-----finished!'.format(i),'!!!!!!')

/data/inspur_disk03/userdir/caoxy/software/anaconda2/lib/python2.7/site-packages/matplotlib/figure.py:2299: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  warnings.warn("This figure includes Axes that are not compatible "


('/data/inspur_disk03/userdir/caoxy/eboss_lya/data/4/10243-58159-381_1.png', '---0 image-----finished!')
('/data/inspur_disk03/userdir/caoxy/eboss_lya/data/4/10272-58172-937_1.png', '---1 image-----finished!')
('/data/inspur_disk03/userdir/caoxy/eboss_lya/data/4/10274-58170-211_1.png', '---2 image-----finished!')
('/data/inspur_disk03/userdir/caoxy/eboss_lya/data/4/10287-58101-813_1.png', '---3 image-----finished!')
('/data/inspur_disk03/userdir/caoxy/eboss_lya/data/4/10429-58145-395_1.png', '---4 image-----finished!')
('/data/inspur_disk03/userdir/caoxy/eboss_lya/data/4/10431-58137-159_1.png', '---5 image-----finished!')
('/data/inspur_disk03/userdir/caoxy/eboss_lya/data/4/10431-58137-392_1.png', '---6 image-----finished!')
('----lines:', ['/data/inspur_disk03/userdir/caoxy/eboss_lya/data/4/10431-58137-392_1.png'])
('/data/inspur_disk03/userdir/caoxy/eboss_lya/data/4/10431-58137-392_2.png', '---7 image-----finished!', '!!!!!!')
('/data/inspur_disk03/userdir/caoxy/eboss_lya/data/4/1043

In [2]:
import copy
import pickle
import gzip
from matplotlib import pyplot as plt
import numpy as np
from scipy.signal import medfilt
import h5py

in_dir = '/data/inspur_disk03/userdir/caoxy/eboss_lya/data/'
lya_hits = pickle.load(gzip.open(in_dir+'lya_hits_cut.pzip'))


In [5]:
plate_list

array([10243, 10272, 10274, ...,  9631,  9631,  9632])

# Note
this work should use the results from cross talk fie(just run all it procudure in notebook file)